In [ ]:
!pip install transformers
!pip install -U -q PyDrive

In [ ]:
!pip install simpletransformers
!pip freeze | grep simpletransformers

English Data

In [3]:
import pandas as pd
df=pd.read_csv('/content/eng_3_train.tsv', delimiter='\t') 
df=df[["text", "category"]]
df = df.dropna()
df_dev=pd.read_csv('/content/eng_3_dev.tsv', delimiter='\t') 
df_dev=df_dev[["text", "category"]]
df_dev = df_dev.dropna()
df_test=pd.read_csv('/content/eng_3_test.tsv', delimiter='\t')
df_test = df_test.rename(columns={'text                        ': 'text'}) 
df_test = df_test.dropna()

In [4]:
tedf=pd.read_csv('/content/tam_eng_train.tsv', delimiter='\t') 
tedf=tedf[["text", "category"]]
tedf = tedf.dropna()
tedf_dev=pd.read_csv('/content/tam_eng_dev.tsv', delimiter='\t') 
tedf_dev=tedf_dev[["text", "category"]]
tedf_dev = tedf_dev.dropna()
tedf_test=pd.read_csv('/content/tam_eng_test.tsv', delimiter='\t') 
tedf_test = tedf_test.dropna()

### Mixing english and tamil translated to english data

In [5]:
train = pd.concat([df, tedf])
valid = pd.concat([df_dev,tedf_dev])
test = pd.concat([tedf_test,df_test])

In [6]:
print(len(train))
print(len(valid))
print(len(test))

5817
1456
1819


In [7]:
concatenated = pd.concat([train, test,valid])

import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

df = concatenated

X = df['text']
y = df['category']

testsplit = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=42)

train_val_idx, test_idx = next(testsplit.split(X, y))

X_train_val, X_test = X.iloc[train_val_idx], X.iloc[test_idx]
y_train_val, y_test = y.iloc[train_val_idx], y.iloc[test_idx]


validsplit = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)

train_idx, val_idx = next(validsplit.split(X_train_val, y_train_val))

X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
y_train, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]

train_df = pd.DataFrame({'text': X_train, 'category': y_train})
val_df = pd.DataFrame({'text': X_val, 'category': y_val})
test_df = pd.DataFrame({'text': X_test, 'category': y_test})

print(f'Training set shape: {train_df.shape}')
print(f'Validation set shape: {val_df.shape}')
print(f'Test set shape: {test_df.shape}')


Training set shape: (5454, 2)
Validation set shape: (1819, 2)
Test set shape: (1819, 2)


In [8]:
print(train_df['category'].value_counts())
print(val_df['category'].value_counts())
print(test_df['category'].value_counts())

Non-anti-LGBT+ content    4711
Homophobic                 598
Transphobic                145
Name: category, dtype: int64
Non-anti-LGBT+ content    1571
Homophobic                 199
Transphobic                 49
Name: category, dtype: int64
Non-anti-LGBT+ content    1571
Homophobic                 200
Transphobic                 48
Name: category, dtype: int64


In [9]:
train_df['category'] = train_df['category'].replace('Non-anti-LGBT+ content', 'ally')
val_df['category'] = val_df['category'].replace('Non-anti-LGBT+ content', 'ally')
test_df['category'] = test_df['category'].replace('Non-anti-LGBT+ content', 'ally')

In [10]:
train_df['category'] = train_df['category'].replace('ally', 0)
train_df['category'] = train_df['category'].replace('Homophobic', 1)
train_df['category'] = train_df['category'].replace('Transphobic', 2)

In [11]:
val_df['category'] = val_df['category'].replace('ally', 0)
val_df['category'] = val_df['category'].replace('Homophobic', 1)
val_df['category'] = val_df['category'].replace('Transphobic', 2)

In [12]:
test_df['category'] = test_df['category'].replace('ally', 0)
test_df['category'] = test_df['category'].replace('Homophobic', 1)
test_df['category'] = test_df['category'].replace('Transphobic', 2)

In [13]:
train_df=train_df[["text", "category"]]
train_df = train_df.rename(columns={'text': 'text', 'category': 'labels'})
val_df=val_df[["text", "category"]]
val_df = val_df.rename(columns={'text': 'text', 'category': 'labels'})
test_df=test_df[["text", "category"]]
test_df = test_df.rename(columns={'text': 'text', 'category': 'labels'})

In [ ]:
!pip install emoji

In [15]:
import emoji
train_df['text'] = train_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
val_df['text'] = val_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
test_df['text'] = test_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

## Training Model with Englis Data

In [16]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 3,
             "max_seq_length": 128,
             "train_batch_size": 16,
             "eval_batch_size": 16,
             "logging_steps": 50,
             "save_steps": 2000,
             "learning_rate": 2e-5}

# Create a ClassificationModel
model = ClassificationModel(
    "xlmroberta", "xlm-roberta-base",
    num_labels=3,
    args=train_args
)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

/usr/local/lib/python3.9/dist-packages/simpletransformers/classification/classification_model.py:486: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  warnings.warn(


In [18]:
model.train_model(train_df)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/341 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/341 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/341 [00:00<?, ?it/s]

(1023, 0.3542897258052204)

Validation Set


In [19]:
from sklearn.metrics import f1_score, accuracy_score

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='macro')
result, model_outputs, wrong_predictions = model.eval_model(val_df, f1=f1_multiclass, acc=accuracy_score)
result

  0%|          | 0/1819 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/114 [00:00<?, ?it/s]

{'mcc': 0.5541623451616025,
 'f1': 0.5207727343094359,
 'acc': 0.8938977460142936,
 'eval_loss': 0.3363601378933118}

Testing With English Data

In [20]:
test_df

,text,labels
547,How u will decide the nature,0
2903,"Interviewer very worse communication, you are ...",0
49,Androgen insentivity syndrome. Females who loo...,0
1480,You are from He'll. ... Let God intervene and ...,0
1191,All transgenders,0
...,...,...
1395,Only the Holy Word of God ...will be written i...,0
1371,Thanks for supporting sir,0
326,"A year ago, when my stepdad was there, he crie...",0
412,bible is unscientific. proven in Supreme Court...,0


In [21]:
test = test_df['text'].tolist()
labels = test_df['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/114 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,Androgen insentivity syndrome. Females who loo...,0,1
1,All the trains can't bear the violence of thes...,2,1
2,Bro it is very bad,1,0
3,Che Kanaravi ..,1,0
4,4th December cyclone will come in chennai,0,1
...,...,...,...
193,Unna patha le vandi vartu,1,0
194,Karigalan Cholan Successor s_sandal woman's_sa...,0,1
195,Bunda,1,0
196,9,2,0


In [22]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.94      0.95      0.95      1571
  homophobic       0.55      0.67      0.60       200
 transphobic       0.00      0.00      0.00        48

    accuracy                           0.89      1819
   macro avg       0.50      0.54      0.52      1819
weighted avg       0.88      0.89      0.88      1819



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Training with english and testing with tamil

In [23]:
o_test=pd.read_csv('/content/eng_tam_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/tam_3_test.tsv', delimiter='\t')
e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [24]:
test_ml = pd.concat([o_test,e_test])

In [25]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [26]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [27]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [28]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/114 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,கஞ்சா மற்றும் THC 'tetrahydrocannabinol' கலவை ...,0,1
1,எனது கருத்துக்கு பதிலளித்த நீங்கள் இருவரும் இத...,1,0
2,மனநோய் என்று நினைக்கிறேன். சென்று மருத்துவரை அ...,1,0
3,அந்த பானங்கள் ஆரோக்கியத்திற்கு நல்லதல்ல என்பதை...,0,1
4,அவர்களை போலீசார் தண்டிக்க வேண்டும்,1,0
...,...,...,...
227,இந்தமாதி நாய்களை கண்ட இடத்தில் சுட்டுக்கொல்லனும்,2,1
228,அருமை பேச்சு .இனிமையான குரல். கேள்விக்கேற்ற பத...,1,0
229,ஏன்டா இப்படி பொய்..அவங்க எப்படா சேலையை தூக்கி ...,0,1
230,இவர்களின் வேதனை மிக கொடுமையானது,0,1


In [29]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.92      0.95      0.94      1579
  homophobic       0.46      0.43      0.44       194
 transphobic       0.00      0.00      0.00        46

    accuracy                           0.87      1819
   macro avg       0.46      0.46      0.46      1819
weighted avg       0.85      0.87      0.86      1819



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Training with english and testing with Hindi

In [30]:
o_test=pd.read_csv('/content/eng_hin_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/tam_hin_test.tsv', delimiter='\t')
e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [31]:
test_ml = pd.concat([o_test,e_test])
len(test_ml)

1819

In [32]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [33]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [34]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [35]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/114 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"संस्कृति की बात करने वालों के लिए, शादी से पहल...",0,1
1,वास्तव में यह शरारत समाज से अधिक शक्तिशाली प्र...,0,1
2,धारा 377 थम्स अप थम्स अप,0,1
3,"आप दोनों जिन्होंने मेरी टिप्पणी का जवाब दिया, ...",1,0
4,ऐसा देखभाल करने वाला व्यक्ति दो_दिलों को लाल_द...,0,1
...,...,...,...
221,किसी लड़के को लड़का नहीं होना चाहिए,0,1
222,इसी महीने कुत्तों को देखते ही गोली मार दी गई,2,1
223,सुंदर वाणी मधुर वाणी। सवाल का जवाब बहुत अच्छा ...,1,0
224,यह लड़की तिरुवन्नामलाई है,0,1


In [36]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.93      0.94      0.94      1579
  homophobic       0.48      0.58      0.52       194
 transphobic       0.00      0.00      0.00        46

    accuracy                           0.88      1819
   macro avg       0.47      0.51      0.49      1819
weighted avg       0.86      0.88      0.87      1819



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Training with english and testing with Telugu

In [37]:
o_test=pd.read_csv('/content/eng_tel_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/tam_tel_test.tsv', delimiter='\t')
e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [38]:
test_ml = pd.concat([o_test,e_test])
len(test_ml)

1819

In [39]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [40]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [41]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [42]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/114 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"సంస్కృతి గురించి మాట్లాడే వారికి, వివాహానికి మ...",0,1
1,మీరు గంజాయి మరియు THC 'tetrahydrocannabinol సమ...,0,1
2,నా వ్యాఖ్యకు ప్రత్యుత్తరం ఇచ్చిన మీరిద్దరూ దీన...,1,0
3,దయచేసి alok ఇవ్వండి దయచేసి దయచేసి దయచేసి దయచేస...,0,1
4,డైరెక్టర్ సర్ ప్లీస్ గే పశ్చాత్తాపపడండి అగురా ...,1,0
...,...,...,...
241,ఆమె చాలా చక్కగా నటిస్తుంది. మంచూర్‌లో డబ్బులు ...,2,0
242,ఏ అబ్బాయి కూడా అబ్బాయిగా ఉండకూడదు,0,1
243,ఈ నెలలో కుక్కలను చూసి కాల్చి చంపారు,2,0
244,చక్కని ప్రసంగం. మధురమైన స్వరం. అన్న ప్రశ్నకు స...,1,0


In [43]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.91      0.94      0.93      1579
  homophobic       0.45      0.45      0.45       194
 transphobic       0.00      0.00      0.00        46

    accuracy                           0.86      1819
   macro avg       0.45      0.46      0.46      1819
weighted avg       0.84      0.86      0.85      1819



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Training with english and testing with Malyalam

In [44]:
o_test=pd.read_csv('/content/eng_ml_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/tam_ml_test.tsv', delimiter='\t')
e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [45]:
test_ml = pd.concat([o_test,e_test])
len(test_ml)

1819

In [46]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [47]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [48]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [49]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/114 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"സംസ്കാരത്തെക്കുറിച്ച് സംസാരിക്കുന്നവർക്ക്, വിവ...",0,1
1,എന്റെ അഭിപ്രായത്തിന് മറുപടി നൽകിയ നിങ്ങൾ രണ്ടു...,1,0
2,എനിക്ക് തോന്നുന്നു .. മാനസിക രോഗമാണ്. പോയി ഡോക...,1,0
3,ആ പാനീയങ്ങൾ ആരോഗ്യത്തിന് ഒട്ടും നല്ലതല്ലെന്ന് ...,0,1
4,പോലീസ് അവരെ ശിക്ഷിക്കണം,1,0
...,...,...,...
214,നിങ്ങൾ ബുദ്ധിപരമായി സംസാരിക്കാൻ വിചാരിക്കുന്നു...,1,0
215,അവൾ വളരെ ഭംഗിയായി അഭിനയിക്കുന്നു. മഞ്ചൂരിൽ സർക...,2,0
216,ഒരു പുരുഷനും ഒരു പുരുഷനാകേണ്ടതില്ല,0,1
217,ഈ മാസം നായ്ക്കളെ കണ്ടപ്പോൾ വെടിവച്ചു,2,1


In [50]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.92      0.96      0.94      1579
  homophobic       0.49      0.43      0.46       194
 transphobic       0.00      0.00      0.00        46

    accuracy                           0.88      1819
   macro avg       0.47      0.46      0.47      1819
weighted avg       0.85      0.88      0.86      1819



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
